### Script to query Each items from BANNER db
* Running this piece of code before starting the analysis significantly reduces the time consuming querying process.
* This script qeuries the BANNER dB for each feature. (Not filtered by list of encounters of interest.)
* Then saves each dataframe in a designated directory.

In [1]:
import pandas as pd
import pickle
import os
import numpy as np

In [2]:
fileDir = os.path.dirname('__file__')

In [3]:
enc_df = pd.read_csv(os.path.join(fileDir, 'csv_banner', 'Banner_encounters.csv'))

In [ ]:
parDir = os.path.join(fileDir, 'BannerData_pandas', '160701_no_strings_has_duplicates')
pdDirs = [x[0] for x in os.walk(parDir)]
pdDirs = pdDirs[1:]

f_dfs = list()
for pdDir in pdDirs:
    f_df = [x[2] for x in os.walk(pdDir)]
    f_df = [os.path.join(pdDir, f) for f in f_df[0]]
    f_dfs.extend(f_df)

** Below cell is to concatenate the 1026 chartevent tables into 11 chart event tables. **
* This is to reduce the time loading each table for querying each feature (item)
* Run the below cell only when you haven't created 11 concatenated chart event tables

In [ ]:
run_cell = False

def concatDF(f_dfs):
    chart_df = pd.DataFrame()
    for f_df in f_dfs:
#         print('Loading {}'.format(f_df))
        ban_df = pd.read_pickle(f_df)
        ban_df = convertDtype(ban_df)
        chart_df = pd.concat([chart_df, ban_df], axis=0)
        
    print('Chart DF number of rows: {}'.format(len(chart_df.index)))
    return chart_df

if run_cell:
    stump_size = 100
    idx_group = np.arange(0, len(f_dfs), stump_size)
    idx_group = np.append(idx_group, len(f_dfs))

    f_group = list()
    for idx_start, idx_end in zip(idx_group[:-1], idx_group[1:]):
        f_group.append(f_dfs[idx_start:idx_end])

    f_name = os.path.join(fileDir, 'BannerData_pandas', 'chart_df')
    count = 0
    for files in f_group:
        f_name_full = f_name+'{:02d}'.format(count)+'.pkl'
        count +=1
        chart_df = concatDF(files)
        chart_df.to_pickle(f_name_full)

** Below cell is to find the item_id for each feature **
* Skip this cell if not necessary

In [5]:
def convertDtype(ban_df):
    ban_df.EVENT_CD = pd.to_numeric(ban_df.EVENT_CD)
    ban_df.RESULT_UNITS_CD = pd.to_numeric(ban_df.RESULT_UNITS_CD)
    return ban_df

def getItemID(ban_df):
    item_id = ban_df.groupby('EVENT_CD')['EVENT_NAME'].unique().to_frame().reset_index()
    item_id.EVENT_NAME = np.hstack(item_id.EVENT_NAME)
    return item_id

import re
def findPattern(item_id, pattern):
#     pattern = re.compile("[Hh]emo")
    match = {row.EVENT_CD: row.EVENT_NAME for idx, row in item_id.iterrows() 
             if pattern.match(row.EVENT_NAME) is not None}
    return match

fBig = os.path.join(fileDir, 'BannerData_pandas', 'chart_df00.pkl')
chartBig_df = pd.read_pickle(fBig)
item_name = getItemID(chartBig_df)

# feature = 'glucose'
# pattern = re.compile('.*[gG]lucose')
# feature = 'albumin'
# pattern = re.compile('.*[aA]lbumin')
# feature = 'bilirubin'
# pattern = re.compile('.*[bB]ilirubin')
# feature = 'bun'
# pattern = re.compile('.*[bB][uU][nN]$')
# feature = 'calcium'
# pattern = re.compile('[cC]alcium$')
# feature = 'creatinine'
# pattern = re.compile('.*[cC]reatinine$')
# feature = 'fio2'
# pattern = re.compile('.*[fF][iI][oO]2$')
# feature = 'hemoglobin'
# pattern = re.compile('.*[hH][eE][mM][oO][gG][lL][oO][bB][iI][nN]')
# feature = 'hr'
# pattern = re.compile('Heart Rate$')
feature = 'lactic_acid'
pattern = re.compile('[lL]actic [aA]cid')
feature = 'pao2'
pattern = re.compile('[pP][oO]2')
match = findPattern(item_name, pattern)
print(match)

In [24]:
item_id_ban = {'albumin': [705919],
               'creatinine': [681994],
               'fio2': [3053636, 826661], 
               'glucose': [15616636], 
               'hemoglobin': [3158492, 463628023], 
               'hr': [3049471], 
               'lactic_acid': [705954,9792755,133416203,245357455,981787332,9792756],
               'map': [100136725, 3049686], 
               'ndbp': [3059689], 
               'nsbp': [3059679], 
               'pao2': [120802110, 15616657, 1035600271], 
               'ph': [15616654,15616653,3158213,706018,120802108],
               'platelet': [682006], 
               'potassium': [681988], 
               'ratio_pao2_flo2': None,
               'sao2': [3053467], 
               'spo2': [3053467],
               'temperature': [3144901], 
               'urine': [3141699,3130213,463812828,3106998,684010,3141701],
               'wbc': [681997], 
               'bun': [681993],
               'bilirubin': [705932], 
               'calcium': [681996], 
               'weight': [3053089]}

In [25]:
item_pedAKI = ['pao2']
# item_pedAKI = ['fio2', 'hemoglobin', 'lactic_acid', 'map', 'ph', 'urine']


# item_pedAKI = ['albumin', 'creatinine', 'fio2', 'glucose', 
#                'hemoglobin', 'hr', 'lactic_acid', 'map', 
#                'ndbp', 'nsbp', 'pao2', 'ph', 'platelet', 'potassium', 
#                'sao2', 'spo2', 'temperature', 'urine', 'wbc',
#                'bun', 'bilirubin', 'calcium', 'weight']
item_dic_pedAKI = {item: item_id_ban[item] for item in item_pedAKI}
item_dic_pedAKI

{'pao2': [120802110, 15616657, 1035600271]}

In [14]:
def getItemDF(item_id):    
    item_df = pd.DataFrame()
    sfx_all = range(11)
    for sfx in sfx_all:
        fname = 'chart_df{:02d}.pkl'.format(sfx)
        fname = os.path.join(fileDir, 'BannerData_pandas', fname)
        chart_df = pd.read_pickle(fname)
        chart_df = convertDtype(chart_df)
        item_df_sub = chart_df.loc[np.in1d(chart_df.EVENT_CD, item_id),:]        
        item_df = pd.concat([item_df, item_df_sub], axis=0)
    return item_df

In [27]:
if not os.path.exists(os.path.join(fileDir, 'item_df_banner')):
        os.makedirs(os.path.join(fileDir, 'item_df_banner'))
        
for item in item_dic_pedAKI:
    item_id = item_dic_pedAKI[item]
    fname = os.path.join(fileDir, 'item_df_banner', 'banner_{}_df.pkl'.format(item))
    if not os.path.exists(fname):
        print('Querying {}'.format(item))
        item_df = getItemDF(item_id)
        item_df.to_pickle(fname)
        print('Queried {}'.format(item))
#         print(item_df.head())
    

Querying pao2
Queried pao2
